In [1]:
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
from scipy.linalg import cholesky, cho_solve
import time

In [2]:
np.random.seed(0)
theta_init = np.array([2., 8.])
x_p_init = np.random.rand(5, 2)
x_q_init = np.random.rand(3, 2)

In [3]:
def scipy_cdist(x_p, x_q, theta):
    return cdist(x_p/theta, x_q/theta, 'sqeuclidean')

In [4]:
scipy_cdist(x_p_init, x_q_init, theta_init)

array([[ 0.01529378,  0.0007842 ,  0.06323125],
       [ 0.00893062,  0.00256608,  0.07395753],
       [ 0.03408281,  0.0064345 ,  0.0359634 ],
       [ 0.03341091,  0.00427266,  0.04370637],
       [ 0.00772122,  0.04372113,  0.2005675 ]])

In [5]:
def numpy_cdist(x_p, x_q, theta):
    z_p = x_p/theta  # (n_p, d)
    z_q = x_q/theta  # (n_q, d)
    d_pq = np.dot(z_p, z_q.T)  # (n_p, n_q)
    d_p = np.sum(z_p ** 2, axis=1)  # (n_p,)
    d_q = np.sum(z_q ** 2, axis=1)  # (n_q,)
    return (d_p + (-2 * d_pq + d_q).T).T

In [6]:
numpy_cdist(x_p_init, x_q_init, theta_init)

array([[ 0.01529378,  0.0007842 ,  0.06323125],
       [ 0.00893062,  0.00256608,  0.07395753],
       [ 0.03408281,  0.0064345 ,  0.0359634 ],
       [ 0.03341091,  0.00427266,  0.04370637],
       [ 0.00772122,  0.04372113,  0.2005675 ]])

In [7]:
def tf_cdist(x_p, x_q, theta):
    z_p = tf.divide(x_p, theta)  # (n_p, d)
    z_q = tf.divide(x_q, theta)  # (n_q, d)
    d_pq = tf.matmul(z_p, tf.transpose(z_q))  # (n_p, n_q)
    d_p = tf.reduce_sum(tf.square(z_p), axis=1)  # (n_p,)
    d_q = tf.reduce_sum(tf.square(z_q), axis=1)  # (n_q,)
#     return tf.transpose(d_p + tf.transpose(-2 * d_pq + d_q))
    return d_p[:, np.newaxis] - 2 * d_pq + d_q
# gamma = tf.constant(-50.0)
# dist = tf.reduce_sum(tf.square(x_data), 1)
# dist = tf.reshape(dist, [-1,1])
# sq_dists = tf.add(tf.subtract(dist, tf.multiply(2., tf.matmul(x_data, tf.transpose(x_data)))), tf.transpose(dist))
# my_kernel = tf.exp(tf.multiply(gamma, tf.abs(sq_dists)))

In [8]:
x_p = tf.placeholder(tf.float64, shape=list(x_p_init.shape))
x_q = tf.placeholder(tf.float64, shape=list(x_q_init.shape))
theta = tf.placeholder(tf.float64, shape=list(theta_init.shape))
tf_gram_matrix = tf_cdist(x_p, x_q, theta)
shape = tf.shape(tf_gram_matrix)
shape[0]

<tf.Tensor 'strided_slice_1:0' shape=() dtype=int32>

In [9]:
sess = tf.Session()
tf_gram_matrix_eval = sess.run(tf_gram_matrix, feed_dict={x_p: x_p_init, x_q: x_q_init, theta: theta_init})
tf_gram_matrix_eval

array([[ 0.01529378,  0.0007842 ,  0.06323125],
       [ 0.00893062,  0.00256608,  0.07395753],
       [ 0.03408281,  0.0064345 ,  0.0359634 ],
       [ 0.03341091,  0.00427266,  0.04370637],
       [ 0.00772122,  0.04372113,  0.2005675 ]])

In [10]:
theta_last = 2 * theta[1:] + 1
sess.run(theta_last, feed_dict={theta: theta_init})

array([ 17.])

In [11]:
theta_check = theta > 0.5
sess.run(theta_check, feed_dict={theta: theta_init})

array([ True,  True], dtype=bool)

In [12]:
theta_new_axis = theta[:, np.newaxis]
sess.run(theta_new_axis, feed_dict={theta: theta_init})

array([[ 2.],
       [ 8.]])

In [13]:
theta_clip = tf.clip_by_value(theta, 5., np.inf)
print(tf.shape(theta_clip))
sess.run(theta_clip, feed_dict={theta: theta_init})

Tensor("Shape_1:0", shape=(1,), dtype=int32)


array([ 5.,  8.])

In [14]:
theta_less = tf.less_equal(theta, 5)
s = tf.range(tf.shape(theta)[0])
print(sess.run(s, feed_dict={theta: theta_init}))
sess.run(theta_less, feed_dict={theta: theta_init})

[0 1]


array([ True, False], dtype=bool)

In [15]:
p = tf.placeholder(tf.float32, shape=[10])
invalid_condition = tf.less_equal(p, 0)
p_adjust = tf.select(invalid_condition, tf.ones(tf.shape(p)), p)
p_init = np.random.rand(10) - 0.5
sess.run(p_adjust, feed_dict={p: p_init})

array([ 1.        ,  0.33261985,  0.27815676,  0.37001213,  0.47861835,
        0.29915857,  1.        ,  0.28052917,  1.        ,  0.13992102], dtype=float32)

In [16]:
ind_val = np.random.randint(0, 3, (10,))
classes_val = np.array([4, 5, 8])
pred_val = classes_val[ind_val]
print(ind_val)
print(classes_val)
print(pred_val)

[1 1 0 2 0 2 2 0 2 0]
[4 5 8]
[5 5 4 8 4 8 8 4 8 4]


In [17]:
sess = tf.Session()
ind = tf.placeholder(tf.int32, shape=[10])
classes = tf.placeholder(tf.int32, shape=[3])
pred = tf.gather(classes, ind, validate_indices=None, name=None)
print(pred)
sess.run(pred, feed_dict={ind: ind_val, classes: classes_val})

Tensor("Gather:0", shape=(10,), dtype=int32)


array([5, 5, 4, 8, 4, 8, 8, 4, 8, 4], dtype=int32)

In [29]:
indices = tf.range(tf.shape(classes)[0])
sess.run(indices, feed_dict={ind: ind_val, classes: classes_val})

array([0, 1, 2], dtype=int32)

In [18]:
variable = tf.placeholder(tf.float64, shape=[4])
theta_new_axis = tf.cast(tf.equal(theta[:, np.newaxis], variable), tf.float64)
sess.run(theta_new_axis, feed_dict={theta: np.array([1, 3]), variable: np.arange(4)})

array([[ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [19]:
def scipy_gaussian(x_p, x_q, theta):
    return np.exp(-0.5 * scipy_cdist(x_p, x_q, theta))

In [20]:
def numpy_gaussian(x_p, x_q, theta):
    return np.exp(-0.5 * numpy_cdist(x_p, x_q, theta))

In [21]:
def tf_gaussian(x_p, x_q, theta):
    return tf.exp(-0.5 * tf_cdist(x_p, x_q, theta))

In [22]:
def scipy_mock_predict(x, y, x_q, hypers):
    start = time.clock()
    n = x.shape[0]
    theta = hypers[:-1]
    zeta = hypers[-1]
    k_reg = scipy_gaussian(x, x, theta) + n * zeta ** 2 * np.eye(n)
    k_q = scipy_gaussian(x, x_q, theta)
    lower = True
    l = cholesky(k_reg, lower=lower, check_finite=False)
    b = cho_solve((l, lower), k_q, check_finite=False)
    time_took = time.clock() - start
    return np.dot(y, b), time_took

In [23]:
def numpy_mock_predict(x, y, x_q, hypers):
    start = time.clock()
    n = x.shape[0]
    theta = hypers[:-1]
    zeta = hypers[-1]
    k_reg = numpy_gaussian(x, x, theta) + n * zeta ** 2 * np.eye(n)
    k_q = numpy_gaussian(x, x_q, theta)
    lower = True
    l = cholesky(k_reg, lower=lower, check_finite=False)
    b = cho_solve((l, lower), k_q, check_finite=False)
    time_took = time.clock() - start
    return np.dot(y, b), time_took

In [24]:
def tf_mock_predict(x, y, x_q, hypers):
    n = tf.shape(x)[0]
    i = tf.cast(tf.eye(n), tf.float64)
    theta = hypers[:-1]
    zeta = hypers[-1]
    k_reg = tf_gaussian(x, x, theta) + tf.multiply(tf.cast(n, tf.float64), tf.multiply(zeta ** 2, i))
    k_q = tf_gaussian(x, x_q, theta)
    lower = True
    l = tf.cholesky(k_reg)
    b = tf.cholesky_solve(l, k_q)
    return tf.matmul(y[np.newaxis, :], b)[0]

In [25]:
def tf_mock_predict_eval(x_eval, y_eval, x_q_eval, hypers_eval):
    x = tf.placeholder(tf.float64, shape=list(x_eval.shape))
    y = tf.placeholder(tf.float64, shape=list(y_eval.shape))
    x_q = tf.placeholder(tf.float64, shape=list(x_q_eval.shape))
    hypers = tf.placeholder(tf.float64, shape=list(hypers_eval.shape))
    result = tf_mock_predict(x, y, x_q, hypers)
    hypers_grad = tf.gradients(result, [hypers])[0]
    sess = tf.Session()
    start = time.clock()
    answer = sess.run(result, feed_dict={x: x_eval, y: y_eval, x_q: x_q_eval, hypers: hypers_eval})
    time_took = time.clock() - start
    start = time.clock()
    answer_grad = sess.run(hypers_grad, feed_dict={x: x_eval, y: y_eval, x_q: x_q_eval, hypers: hypers_eval})
    time_took_grad = time.clock() - start
    return answer, answer_grad, time_took, time_took_grad

In [26]:
np.random.seed(100)
n = 2000
n_q = 5
d = 20
x = np.random.rand(n, d)
w = np.random.rand(d)
b = -1
y = np.sum(w * x + b, axis=1)
print(y)
x_q = np.random.rand(n_q, d)
hypers = np.append(np.random.rand(d), 1e-4)
print(hypers.shape)

[-16.2800275  -15.91239191 -16.49401066 ..., -16.32422978 -15.76398735
 -15.86634023]
(21,)


In [27]:
numpy_result = numpy_mock_predict(x, y, x_q, hypers)
print('Numpy Done.')
scipy_result = scipy_mock_predict(x, y, x_q, hypers)
print('Scipy Done.')
tf_result = tf_mock_predict_eval(x, y, x_q, hypers)
print("Tensorflow Done.")
np.set_printoptions(2)
print(numpy_result)
print(scipy_result)
print(tf_result)

Numpy Done.
Scipy Done.
Tensorflow Done.
(array([-0.28, -0.  , -0.14, -0.  , -0.2 ]), 1.4319080000000004)
(array([-0.28, -0.  , -0.14, -0.  , -0.2 ]), 1.0324749999999998)
(array([-0.28, -0.  , -0.14, -0.  , -0.2 ]), array([ -8.45e-02,  -7.52e-01,  -9.95e-02,  -3.61e-02,  -2.86e+01,
        -4.76e-01,  -1.65e+00,  -1.06e-01,  -7.55e+00,  -8.31e-02,
        -2.58e+00,  -9.47e-02,  -2.42e-02,  -5.52e+00,  -2.62e-02,
        -2.55e-02,  -5.60e-02,  -6.33e-01,  -1.63e+01,  -1.23e+00,
         2.52e-01]), 1.1604520000000003, 2.491683)


In [28]:
# from timeit import timeit
# print(timeit('numpy_mock_predict(x, y, x_q, hypers)', number=10000, setup="from __main__ import numpy_mock_predict, x, y, x_q, hypers"))
# print(timeit('scipy_mock_predict(x, y, x_q, hypers)', number=10000, setup="from __main__ import scipy_mock_predict, x, y, x_q, hypers"))
# print(timeit('tf_mock_predict_eval(x, y, x_q, hypers)', number=1, setup="from __main__ import tf_mock_predict_eval, x, y, x_q, hypers"))